In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('/scratch/gpfs/jiaxuanl/Data/popsed/')
import sys
sys.path.append('/home/jiaxuanl/Research/popsed/')

In [41]:
import numpy as np
import torch
from torch import Tensor, FloatTensor, optim
import faiss.contrib.torch_utils

In [274]:
# phot_all = np.load('/scratch/gpfs/jiaxuanl/Data/popsed/train_sed/phot_all.npy')
# X = -2.5 * np.log10(phot_all).astype('float32')
# X.shape

In [275]:
import faiss
from scipy.special import digamma

In [276]:
## Below, I try to compare re-write KNN using faiss

In [278]:
d = 5
nb = 10000                      # database size
np.random.seed(1234)
X = np.random.random((nb, d)).astype('float32')
np.random.seed(4321)
Y = np.random.random((nb, d)).astype('float32')

Y[:, 0] += 1

X = FloatTensor(X)
Y = FloatTensor(Y)

In [285]:
n, d = X.shape  # X sample size, dimensions

In [286]:
NN_X = faiss.IndexFlatL2(d)   # build the index
NN_X.add(X)                  # add vectors to the index

In [263]:
assert X.shape[1] == Y.shape[1]
n, d = X.shape  # X sample size, dimensions
m = Y.shape[0]  # Y sample size

In [264]:
NN_X = faiss.IndexFlatL2(d)   # build the index
NN_X.add(X)                  # add vectors to the index

NN_Y = faiss.IndexFlatL2(d)   # build the index
NN_Y.add(Y)                  # add vectors to the index

In [254]:
dNN1_XX, ind_XX = NN_X.search(X, 2000)
dNN1_XX = torch.sqrt(dNN1_XX[:, 1:])
ind_XX = ind_XX[:, 1:]

dNN1_XY, ind_XY = NN_Y.search(X, 2000)
dNN1_XY = torch.sqrt(dNN1_XY)

In [255]:
eps = torch.amax(torch.cat((dNN1_XX[:, 0:1], dNN1_XY[:, 0:1]), dim=1), 1) * 1.000001
eps = eps.type(torch.float64)
#eps = eps.astype('float64')
print('  epsilons ', eps)

  epsilons  tensor([0.1424, 0.1324, 0.1013,  ..., 0.1150, 0.1354, 0.1193],
       dtype=torch.float64)


In [256]:
# find l_i and k_i, slow
l_i = torch.empty(n, dtype=int)
k_i = torch.empty(n, dtype=int)
rho_i = torch.empty(n, dtype=float)
nu_i = torch.empty(n, dtype=float)

for i, e in enumerate(eps):
    l_i[i] = torch.sum(dNN1_XX[i:i+1] < e)
    rho_i[i] = dNN1_XX[i:i+1][0][l_i[i] - 1]
    
    k_i[i] = torch.sum(dNN1_XY[i:i+1] < e)
    nu_i[i] = dNN1_XY[i:i+1][0][k_i[i] - 1]

In [257]:
d_corr = d / n * torch.sum(torch.log(nu_i / rho_i))
#if not silent:
print('  first term = %f' % d_corr)
digamma_term = torch.sum(digamma(l_i) - digamma(k_i)) / n
#if not silent:
print('  digamma term = %f' % digamma_term)
d_corr + digamma_term + np.log(float(m)/float(n-1))

  first term = -0.020082
  digamma term = -0.009040


tensor(-0.0290, dtype=torch.float64)

In [260]:
from popsed.nde import KL_w2009_eq29, _KL_w2009_eq29, diff_KL_w2009_eq29

In [284]:
%%time
diff_KL_w2009_eq29(X, Y, silent=False)

  epsilons  tensor([0.8452, 0.7576, 0.6938,  ..., 0.5891, 0.6466, 0.4214],
       dtype=torch.float64)
  l_i  tensor([999, 999, 999,  ..., 999, 999, 608])
  k_i  tensor([1, 1, 1,  ..., 1, 1, 1])
  first term = 0.827287
  digamma term = 6.001824
CPU times: user 4.27 s, sys: 89.8 ms, total: 4.36 s
Wall time: 4.4 s


tensor(6.8292, dtype=torch.float64)

In [267]:
%%time
KL_w2009_eq29(X, Y, silent=False)

  epsilons  [0.84527129 0.75766706 0.69386774 ... 0.58919084 0.64670169 0.42141047]
  l_i  [5615 2157 4608 ... 1722 2669  608]
  k_i  [1 1 1 ... 1 1 1]
  first term = 0.037714
  digamma term = 6.564656
CPU times: user 2.39 s, sys: 4.01 ms, total: 2.4 s
Wall time: 2.41 s


6.602469623349827

In [268]:
%%time
_KL_w2009_eq29(X, Y, silent=True)

CPU times: user 13.8 s, sys: 157 ms, total: 13.9 s
Wall time: 13.9 s


6.605654392964036

In [120]:
## we already have eps

In [206]:
%%time
l_i = torch.empty(n, dtype=int)
k_i = torch.empty(n, dtype=int)
rho_i = torch.empty(n, dtype=float)
nu_i = torch.empty(n, dtype=float)

for i, e in enumerate(eps):
    dist = torch.norm(X[i:i+1] - X, dim=1, p=None).sort()
    l_i[i] = torch.sum(dist.values <= e) - 1
    rho_i[i] = dist.values[l_i[i]]
    
    dist = torch.norm(X[i:i+1] - Y, dim=1, p=None).sort()
    k_i[i] = torch.sum(dist.values <= e)
    nu_i[i] = dist.values[k_i[i] - 1]

CPU times: user 48.6 s, sys: 4.87 ms, total: 48.6 s
Wall time: 48.6 s


In [144]:
l_i -= 1

In [214]:
assert rho_i.min() > 0., 'duplicate elements in your chain'

d_corr = d / n * torch.sum(torch.log(nu_i / rho_i))

print('  first term = %f' % d_corr)
digamma_term = torch.sum(digamma(l_i) - digamma(k_i)) / n

print('  digamma term = %f' % digamma_term)

  first term = -0.000908
  digamma term = -0.820392


In [146]:
d_corr + digamma_term + np.log(float(m)/float(n-1))

tensor(-0.0293, dtype=torch.float64)

## SKlearn method

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [133]:
n, d = X.shape  # X sample size, dimensions
m = Y.shape[0]  # Y sample size

In [134]:
NN_X = NearestNeighbors(n_neighbors=1).fit(X)
NN_Y = NearestNeighbors(n_neighbors=1).fit(Y)

In [135]:
dNN1_XX, ind_XX = NN_X.kneighbors(X, n_neighbors=2)
dNN1_XY, ind_XY = NN_Y.kneighbors(X)

In [136]:
eps = np.amax([dNN1_XX[:, 1], dNN1_XY[:, 0]], axis=0) * 1.000001
print('  epsilons ', eps)

  epsilons  [0.14238908 0.13238188 0.10129241 ... 0.1149996  0.1353781  0.11926264]


In [137]:
# find l_i and k_i
_, i_l = NN_X.radius_neighbors(X, eps)
_, i_k = NN_Y.radius_neighbors(X, eps)
l_i = np.array([len(il)-1 for il in i_l])
k_i = np.array([len(ik) for ik in i_k])
# assert l_i.min() > 0
# assert k_i.min() > 0

print('  l_i ', l_i)
print('  k_i ', k_i)

  l_i  [4 2 1 ... 2 1 1]
  k_i  [1 1 1 ... 1 1 1]


In [60]:
rho_i = np.empty(nb, dtype=float)
nu_i = np.empty(nb, dtype=float)
for i in range(nb):
    rho_ii, _ = NN_X.kneighbors(np.atleast_2d(X[i]), n_neighbors=l_i[i]+1)
    nu_ii, _ = NN_Y.kneighbors(np.atleast_2d(X[i]), n_neighbors=k_i[i])
    rho_i[i] = rho_ii[0][-1]
    nu_i[i] = nu_ii[0][-1]

In [64]:
assert rho_i.min() > 0., 'duplicate elements in your chain'

d_corr = float(d) / float(n) * np.sum(np.log(nu_i/rho_i))

print('  first term = %f' % d_corr)
digamma_term = np.sum(digamma(l_i) - digamma(k_i)) / float(n)

print('  digamma term = %f' % digamma_term)

  first term = -0.019828
  digamma term = -0.009140


In [65]:
d_corr, digamma_term, np.log(float(m)/float(n-1))

(-0.019827795680685836, -0.009139642857142857, 0.00010000500033327544)